In [1]:
import tensorflow as tf
import numpy as np

# 使用 NumPy 生成假数据(phony data), 总共 100 个点.
x_data = np.float32(np.random.rand(2, 100)) # 随机输入
y_data = np.dot([0.100, 0.200], x_data) + 0.300

# 构造一个线性模型
# 
b = tf.Variable(tf.zeros([1]))
W = tf.Variable(tf.random_uniform([1, 2], -1.0, 1.0))
y = tf.matmul(W, x_data) + b

# 最小化方差
loss = tf.reduce_mean(tf.square(y - y_data))
optimizer = tf.train.GradientDescentOptimizer(0.5)
train = optimizer.minimize(loss)

# 初始化变量
init = tf.initialize_all_variables()

# 启动图 (graph)
sess = tf.Session()
sess.run(init)

# 拟合平面
for step in range(0, 201):
    sess.run(train)
    if step % 20 == 0:
        print (step, sess.run(W), sess.run(b))

/usr/local/miniconda3/envs/dl/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Use `tf.global_variables_initializer` instead.
0 [[-0.20146708  0.80983573]] [0.34215036]
20 [[0.04614187 0.29653376]] [0.280538]
40 [[0.09375869 0.21796922]] [0.29426846]
60 [[0.09997188 0.20369442]] [0.29812226]
80 [[0.10037021 0.20087546]] [0.29934502]
100 [[0.10020063 0.20024227]] [0.29976404]
120 [[0.10008495 0.2000761 ]] [0.29991364]
140 [[0.10003322 0.20002592]] [0.29996818]
160 [[0.10001261 0.20000923]] [0.29998824]
180 [[0.1000047  0.20000333]] [0.29999566]
200 [[0.10000175 0.20000121]] [0.29999837]


In [1]:
import tensorflow as tf
import numpy as np
import sklearn.preprocessing as prep
from tensorflow.examples.tutorials.mnist import input_data


def xaver_init(fan_in, fan_out, constant=1):
    # 创造一个权重均值为0方差为2/(nin+nout)。
    # 深度学习的权重如果选的小，则信号在每层间的传递会弱。
    # 如果权重取得过大，信号在传递中会被放大而发散，甚至失效。

    low = -constant * np.sqrt(6.0 / (fan_in + fan_out))
    high = constant * np.sqrt(6.0 / (fan_in + fan_out))
    print("low,high", low, high)
    return tf.random_uniform((fan_in, fan_out), minval=low, maxval=high, dtype=tf.float32)

# xaver_init(1,2,1)
class AdditiveGaussianNoiseAutoEncoder(object):
    def __init__(self, n_input, n_hidden, transfer_function=tf.nn.softplus  # 激活函数
                 , optimizer=tf.train.AdamOptimizer(), scale=0.1):
        self.n_input = n_input
        self.n_hidden = n_hidden
        self.transfer = transfer_function
        self.scale = tf.placeholder(tf.float32)  # 预先占位
        self.training_scale = scale
        network_weights = self._initialize_weights()  # 这个是什么？
        self.weights = network_weights

        # 创建网络
        self.x = tf.placeholder(tf.float32, [None, self.n_input])
        #step 1 ： 输入加噪声。
        noise_input_data = self.x + scale*tf.random_normal((n_input,))
        #step 2: y = wx+b

        self.n_hidden = self.transfer(tf.add(tf.matmul(noise_input_data, self.weights['w1']), self.weights['b1']))
        self.reconstruction = tf.add( tf.matmul(self.n_hidden, self.weights['w2']), self.weights['b2'])

        # 创建损失函数
        # x2
        self.cost = tf.reduce_sum(tf.pow(tf.subtract(self.reconstruction,self.x),2))
        self.optimizer = optimizer.minimize(self.cost)
        init = tf.global_variables_initializer()
        self.sess = tf.Session()
        self.sess.run(init)

    def _initialize_weights(self):
        all_weights = dict()
        all_weights['w1'] = tf.Variable(xaver_init(self.n_input,self.n_hidden))
        all_weights['b1'] = tf.Variable(tf.zeros([self.n_hidden]),tf.float32)
        all_weights['w2'] = tf.Variable(tf.zeros([self.n_hidden,self.n_input],tf.float32))
        all_weights['b2'] = tf.Variable(tf.zeros([self.n_input],tf.float32))
        return all_weights

    def partial_fit(self,X):
        cost, opt = self.sess.run((self.cost,self.optimizer),feed_dict={self.x:X,self.scale:self.training_scale})
        return cost

    def calc_total_cost(self,X):
        return self.sess.run(self.cost, feed_dict={self.x:X, self.scale:self.training_scale})

    def tranform(self,X):
        return self.sess.run(self.n_hidden, feed_dict={self.x: X, self.scale: self.training_scale})

    def generate(self,hidden=None):
        if hidden is None:
            hidden = np.random.normal(size=self.weights["b1"])
        return self.sess.run(self.reconstruction,feed_dict={self.n_hidden:hidden})
    def reconstrct(self,X):
        return self.sess.run(self.reconstruction, feed_dict={self.x: X, self.scale: self.training_scale})

    def getWeights(self):
        return self.sess.run(self.weights['w1'])

    def getBiases(self):
        return self.sess.run(self.weights['b1'])

def standard_scale(X_train, X_test):
    preprocessor = prep.StandardScaler().fit(X=X_train) #StandardScaler()要加上括号的
    X_train = preprocessor.transform(X_train)
    X_test  = preprocessor.transform(X_test)
    return X_train, X_test

def get_random_block_from_data(data,batch_size):
    start_index = np.random.randint(0,len(data)-batch_size)
    return data[start_index:(start_index+batch_size)]

if __name__ == '__main__':
    mnist = input_data.read_data_sets('MNIST_data', one_hot=True)
    X_train, X_test = standard_scale(mnist.train.images, mnist.test.images)
    n_samples = int(mnist.train.num_examples)
    training_epochs = 20
    batch_size = 128
    display_step = 1
    autoEncoder = AdditiveGaussianNoiseAutoEncoder(n_input=784, n_hidden=200,
                                     transfer_function=tf.nn.softplus,
                                     optimizer=tf.train.AdamOptimizer(learning_rate=0.01),
                                     scale=0.01)

    for epoch in range(training_epochs):
        avg_cost = 0
        total_batch = int(n_samples/batch_size)
        for i in range(total_batch):
            batch_xs = get_random_block_from_data(X_train,batch_size)
            cost = autoEncoder.partial_fit(batch_xs)
            avg_cost += cost/n_samples*batch_size # 不是很懂

        if epoch % display_step == 0:
            print("epoch:",epoch+1)
            print("cost:", "{:.9f}".format(avg_cost))



/usr/local/miniconda3/envs/dl/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Use the retry module or similar alternatives.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.


URLError: <urlopen error [Errno 99] Cannot assign requested address>